# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import re
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, make_scorer, fbeta_score, f1_score
from sklearn.model_selection import GridSearchCV
from scipy.stats.mstats import gmean
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM df1', con = engine)
X = df [['id', 'message', 'original','genre']]
Y = df [['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone',
 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
 'storm', 'fire', 'earthquake', 'cold', 'other_weather','direct_report']]

In [30]:
X = df['message']
Y = df.iloc[:,4:]

In [31]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:
def pipeline():
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf',MultiOutputClassifier(RandomForestClassifier())),
        ])
    return pipeline

In [33]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [35]:
X_train.head()

6453     mistakes, you can restart has zero. GOD create...
2350     I live in the zone called Karade south of the ...
15364    Last year, China suffered some of the worst fl...
20185    In Sri Lanka, 400,000 people are benefitingfro...
9371     I would like to know where can I do again my c...
Name: message, dtype: object

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [36]:
# train classifier
model=[]
model=pipeline()
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [37]:
Y_pred = model.predict(X_test)
category_names = list(Y)
 # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.65      0.34      0.45      1329
          1       0.80      0.94      0.86      3879
          2       0.50      0.11      0.18        36

avg / total       0.76      0.78      0.75      5244

Accuracy of                   related: 0.78
Category: request 
              precision    recall  f1-score   support

          0       0.89      0.99      0.93      4364
          1       0.84      0.38      0.52       880

avg / total       0.88      0.88      0.87      5244

Accuracy of                   request: 0.88
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.73      0.88  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
multi_f1 = multioutput_fscore(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.25% 

F1 score (custom definition) 92.37%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
def build_model():
    '''
    Build a ML pipeline using ifidf, random forest, and gridsearch
    Input: None
    Output:
        Results of GridSearchCV
    '''
    pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])

    parameters = {
                  'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__min_samples_split': [2, 3, 4],
                  #'clf__estimator__criterion': ['entropy', 'gini']
                 }
    
    #scorer = make_scorer(multioutput_fscore,greater_is_better = True)
   
    #cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2)

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [16]:
model=build_model()
model.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
Y_pred = model.predict(X_test)
    
    # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.73      0.26      0.39      1223
          1       0.81      0.97      0.88      3984
          2       0.67      0.05      0.10        37

avg / total       0.79      0.80      0.76      5244

Accuracy of                   related: 0.80
Category: request 
              precision    recall  f1-score   support

          0       0.90      0.98      0.94      4351
          1       0.86      0.44      0.58       893

avg / total       0.89      0.89      0.88      5244

Accuracy of                   request: 0.89
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.75      0.89  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [27]:
multi_f1 = multioutput_fscore(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.46% 

F1 score (custom definition) 92.60%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Adaboost
#Gradient Boosting
#Starting Verb Extractor

In [54]:
def build_model():
    '''
    Build a ML pipeline using ifidf, random forest, and gridsearch
    Input: None
    Output:
        Results of GridSearchCV
    '''
    pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                        ])

    parameters = {
                  'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__learning_rate': [0.01,0.1],
                 }
    
    #scorer = make_scorer(multioutput_fscore,greater_is_better = True)
   
    #cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2)

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv



In [55]:
Y_pred = model.predict(X_test)
    
    # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))


Category: related 
              precision    recall  f1-score   support

          0       0.65      0.34      0.45      1329
          1       0.80      0.94      0.86      3879
          2       0.50      0.11      0.18        36

avg / total       0.76      0.78      0.75      5244

Accuracy of                   related: 0.78
Category: request 
              precision    recall  f1-score   support

          0       0.89      0.99      0.93      4364
          1       0.84      0.38      0.52       880

avg / total       0.88      0.88      0.87      5244

Accuracy of                   request: 0.88
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.73      0.88  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [56]:
multi_f1 = multioutput_fscore(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.25% 

F1 score (custom definition) 92.37%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [57]:
def build_model():
    '''
    Build a ML pipeline using ifidf, random forest, and gridsearch
    Input: None
    Output:
        Results of GridSearchCV
    '''
    pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
                        ])

    parameters = {
                  'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__learning_rate': [0.01,0.1],
                 }
    
    scorer = make_scorer(multioutput_fscore,greater_is_better = True)
   
    #cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2)

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv



In [58]:
Y_pred = model.predict(X_test)
    
    # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.65      0.34      0.45      1329
          1       0.80      0.94      0.86      3879
          2       0.50      0.11      0.18        36

avg / total       0.76      0.78      0.75      5244

Accuracy of                   related: 0.78
Category: request 
              precision    recall  f1-score   support

          0       0.89      0.99      0.93      4364
          1       0.84      0.38      0.52       880

avg / total       0.88      0.88      0.87      5244

Accuracy of                   request: 0.88
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5229
          1       0.00      0.00      0.00        15

avg / total       0.99      1.00      1.00      5244

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.73      0.88  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
multi_f1 = multioutput_fscore(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.25% 

F1 score (custom definition) 92.37%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.